In [1]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
import mne

import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery, MotorImagery  


moabb.set_log_level("info")
warnings.filterwarnings("ignore")


<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\ABC\anaconda3\envs\py38\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from mne.decoding import CSP
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np

# Récupérer les données
X, labels, meta = paradigm.get_data(dataset=dataset, subjects=[1])

# Convertir meta['session'] en array numpy pour une manipulation facile
sessions = np.array(meta['session'])

# Séparer les indices pour les sessions d'entraînement et de test
train_indices = np.where(sessions == '0train')[0]
test_indices = np.where(sessions == '1test')[0]

# Diviser les données en ensembles d'entraînement et de test
X_train, labels_train = X[train_indices], labels[train_indices]
X_test, labels_test = X[test_indices], labels[test_indices]

print("X_train shape:", X_train.shape)
print("labels_train shape:", labels_train.shape)
print("X_test shape:", X_test.shape)
print("labels_test shape:", labels_test.shape)

# Créer le pipeline avec CSP et LDA
csp = CSP(n_components=8, reg=None, log=True, cov_est='epoch')
lda = LDA()

pipeline = Pipeline([('CSP', csp), ('LDA', lda)])

# Entraîner le modèle
pipeline.fit(X_train, labels_train)

# Prédire sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Calculer les métriques
accuracy = accuracy_score(labels_test, y_pred)
precision = precision_score(labels_test, y_pred, average='macro')
recall = recall_score(labels_test, y_pred, average='macro')
f1 = f1_score(labels_test, y_pred, average='macro')
conf_matrix = confusion_matrix(labels_test, y_pred)
class_report = classification_report(labels_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


X_train shape: (288, 22, 1001)
labels_train shape: (288,)
X_test shape: (288, 22, 1001)
labels_test shape: (288,)
Accuracy: 0.74
Precision: 0.76
Recall: 0.74
F1 Score: 0.74
Confusion Matrix:
[[50  2  0 20]
 [ 3 60  4  5]
 [ 8 16 47  1]
 [15  1  0 56]]
Classification Report:
              precision    recall  f1-score   support

        feet       0.66      0.69      0.68        72
   left_hand       0.76      0.83      0.79        72
  right_hand       0.92      0.65      0.76        72
      tongue       0.68      0.78      0.73        72

    accuracy                           0.74       288
   macro avg       0.76      0.74      0.74       288
weighted avg       0.76      0.74      0.74       288



In [ ]:
from pyriemann.estimation import Covariances
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
from pyriemann.tangentspace import TangentSpace

# Récupérer les données
X, labels, meta = paradigm.get_data(dataset=dataset, subjects=[1])

# Convertir meta['session'] en array numpy pour une manipulation facile
sessions = np.array(meta['session'])

# Séparer les indices pour les sessions d'entraînement et de test
train_indices = np.where(sessions == '0train')[0]
test_indices = np.where(sessions == '1test')[0]

# Diviser les données en ensembles d'entraînement et de test
X_train, labels_train = X[train_indices], labels[train_indices]
X_test, labels_test = X[test_indices], labels[test_indices]

print("X_train shape:", X_train.shape)
print("labels_train shape:", labels_train.shape)
print("X_test shape:", X_test.shape)
print("labels_test shape:", labels_test.shape)

# Créer le pipeline avec Covariances, TangentSpace et SVM
cov_estimator = Covariances(estimator='lwf')
tangent_space = TangentSpace()
scaler = StandardScaler()
svm = SVC(kernel='linear')

pipeline = Pipeline([
    ('Covariances', cov_estimator),
    ('TangentSpace', tangent_space),
    ('Scaler', scaler),
    ('SVM', svm)
])

# Entraîner le modèle
pipeline.fit(X_train, labels_train)

# Prédire sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Calculer les métriques
accuracy = accuracy_score(labels_test, y_pred)
precision = precision_score(labels_test, y_pred, average='macro')
recall = recall_score(labels_test, y_pred, average='macro')
f1 = f1_score(labels_test, y_pred, average='macro')
conf_matrix = confusion_matrix(labels_test, y_pred)
class_report = classification_report(labels_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


X_train shape: (288, 22, 1001)
labels_train shape: (288,)
X_test shape: (288, 22, 1001)
labels_test shape: (288,)
Accuracy: 0.76
Precision: 0.77
Recall: 0.76
F1 Score: 0.76
Confusion Matrix:
[[52  1  0 19]
 [ 2 50 15  5]
 [ 5  5 62  0]
 [16  0  0 56]]
Classification Report:
              precision    recall  f1-score   support

        feet       0.69      0.72      0.71        72
   left_hand       0.89      0.69      0.78        72
  right_hand       0.81      0.86      0.83        72
      tongue       0.70      0.78      0.74        72

    accuracy                           0.76       288
   macro avg       0.77      0.76      0.76       288
weighted avg       0.77      0.76      0.76       288



In [ ]:
from pyriemann.estimation import Covariances
from pyriemann.classification import MDM
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np

# Récupérer les données
X, labels, meta = paradigm.get_data(dataset=dataset, subjects=[1])

# Convertir meta['session'] en array numpy pour une manipulation facile
sessions = np.array(meta['session'])

# Séparer les indices pour les sessions d'entraînement et de test
train_indices = np.where(sessions == '0train')[0]
test_indices = np.where(sessions == '1test')[0]

# Diviser les données en ensembles d'entraînement et de test
X_train, labels_train = X[train_indices], labels[train_indices]
X_test, labels_test = X[test_indices], labels[test_indices]

print("X_train shape:", X_train.shape)
print("labels_train shape:", labels_train.shape)
print("X_test shape:", X_test.shape)
print("labels_test shape:", labels_test.shape)

# Créer le pipeline avec Covariances et MDM
cov_estimator = Covariances(estimator='lwf')
mdm = MDM()

pipeline = Pipeline([('Covariances', cov_estimator), ('MDM', mdm)])

# Entraîner le modèle
pipeline.fit(X_train, labels_train)

# Prédire sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Calculer les métriques
accuracy = accuracy_score(labels_test, y_pred)
precision = precision_score(labels_test, y_pred, average='macro')
recall = recall_score(labels_test, y_pred, average='macro')
f1 = f1_score(labels_test, y_pred, average='macro')
conf_matrix = confusion_matrix(labels_test, y_pred)
class_report = classification_report(labels_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


X_train shape: (288, 22, 1001)
labels_train shape: (288,)
X_test shape: (288, 22, 1001)
labels_test shape: (288,)
Accuracy: 0.70
Precision: 0.71
Recall: 0.70
F1 Score: 0.71
Confusion Matrix:
[[40  2  1 29]
 [ 7 55  9  1]
 [ 5 10 57  0]
 [20  1  0 51]]
Classification Report:
              precision    recall  f1-score   support

        feet       0.56      0.56      0.56        72
   left_hand       0.81      0.76      0.79        72
  right_hand       0.85      0.79      0.82        72
      tongue       0.63      0.71      0.67        72

    accuracy                           0.70       288
   macro avg       0.71      0.70      0.71       288
weighted avg       0.71      0.70      0.71       288



In [ ]:
print(paradigm.get_data(dataset=dataset, subjects=[1])[2])

     subject session run
0          1  0train   0
1          1  0train   0
2          1  0train   0
3          1  0train   0
4          1  0train   0
..       ...     ...  ..
571        1   1test   5
572        1   1test   5
573        1   1test   5
574        1   1test   5
575        1   1test   5

[576 rows x 3 columns]


In [ ]:
dataset = BNCI2014_001()
dataset.subject_list = [1]
paradigm = MotorImagery()
pipeline = make_pipeline(CSP(n_components=8), LDA())
evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=[dataset],
    overwrite=True,
    hdf5_path=None,
)
results = evaluation.process({"csp+lda": pipeline})

2024-07-10 12:47:43,664 WARNING MainThread moabb.paradigms.motor_imagery Choosing from all possible events
2024-07-10 12:47:43,677 INFO MainThread moabb.evaluations.base Processing dataset: BNCI2014-001
BNCI2014-001-WithinSession:   0%|          | 0/1 [00:00<?, ?it/s]

No hdf5_path provided, models will not be saved.


2024-07-10 12:48:05,732 INFO MainThread moabb.evaluations.base csp+lda | BNCI2014-001 | 1 | 0train: Score 0.754


No hdf5_path provided, models will not be saved.


2024-07-10 12:48:15,600 INFO MainThread moabb.evaluations.base csp+lda | BNCI2014-001 | 1 | 1test: Score 0.711
BNCI2014-001-WithinSession: 100%|██████████| 1/1 [00:32<00:00, 32.58s/it]


In [ ]:
print(X.shape)
print(len(labels))

(576, 22, 1001)
576
